In [1357]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2215]:
df = pd.read_csv('default_data_all_variables.csv')
target = pd.read_csv('targets.csv')

In [2204]:
df = df.loc[:, ~df.columns.str.contains('HomeTeam|AwayTeam')]

In [2205]:
df = df.loc[:, ~df.columns.str.contains('year|month')]

In [2209]:
df = df.loc[:, ~df.columns.str.contains('ovr')]

In [2211]:
df[['home_win_odds','draw_odds','away_win_odds']] = target[['home_win_odds','draw_odds','away_win_odds']]

In [1616]:
target = target[['FTHG', 'FTAG']]

In [2213]:
X_train = df[:-1500]
y_train = target[:-1500]
X_val = df[-1500:-760]
y_val = target[-1500:-760]
X_test = df[-760:-10]
y_test = target[-760:-10]
X_to_pred = df[-10:]

In [1588]:
df.to_csv('dataset.csv', index = False)

In [2080]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0005)
model = Sequential()

# Input layer
model.add(Dense(256, activation='relu', input_shape=(182,)))  # 278 is the number of input features
# model.add(Dropout(0.1))
# Hidden layers
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))  # Optional, to prevent overfitting
# model.add(Dense(138, activation='relu'))
# model.add(Dropout(0.1))  # Optional, to prevent overfitting
model.add(Dense(48, activation='relu'))

# Output layer (with softmax to ensure sum-to-1 constraint)
model.add(Dense(2, activation='relu'))

# Compile the model
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
class_weights = {'home_win_odds': 1.0, 'draw_odds': 1, 'away_win_odds': 1.0}
# Train the model
history = model.fit(X_train, y_train, epochs=600, batch_size=2000, validation_data=(X_val, y_val), class_weight=class_weights, callbacks=[early_stopping])
# , callbacks=[early_stopping]
# Evaluate the model
results = model.evaluate(X_test, y_test)

Epoch 1/600


C:\Users\ciesl\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 2.4665 - mae: 1.1495 - val_loss: 1.7417 - val_mae: 1.0456
Epoch 2/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.7041 - mae: 1.0075 - val_loss: 1.5841 - val_mae: 0.9903
Epoch 3/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.6146 - mae: 0.9636 - val_loss: 1.5618 - val_mae: 0.9844
Epoch 4/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5636 - mae: 0.9593 - val_loss: 1.5293 - val_mae: 0.9544
Epoch 5/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5279 - mae: 0.9373 - val_loss: 1.5053 - val_mae: 0.9414
Epoch 6/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4644 - mae: 0.9219 - val_loss: 1.4782 - val_mae: 0.9256
Epoch 7/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4508 - mae: 0.9217 - val_loss: 1.4496 - val_mae: 0.9114
Epoch 8/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4175 - mae: 0.9109 - val_loss: 1.4321 - val_mae: 0.9035
Epoch 9/600
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.4110 - mae: 0.9015 - val_loss: 1

In [2135]:
# model.save("model.keras")
from tensorflow.keras.models import load_model

model = load_model("model.keras")

C:\Users\ciesl\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [2137]:
results

[1.4216986894607544, 0.9110156297683716]

In [2139]:
y_pred = model.predict(X_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [2217]:
y_pred = model.predict(X_to_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [2219]:
y_pred

array([[0.9144526 , 1.7598536 ],
       [2.3543558 , 0.7963938 ],
       [1.7110677 , 0.9329751 ],
       [1.4834449 , 1.3888981 ],
       [2.667153  , 0.9592191 ],
       [3.1310363 , 0.8876346 ],
       [2.0073798 , 0.79347336],
       [1.1977884 , 1.6762311 ],
       [1.4444623 , 1.6478218 ],
       [1.3243672 , 1.3738838 ]], dtype=float32)

In [2221]:
def choose_value2(row, val_draw):
    if row['FTHG']-val_draw > row['FTAG']:
        return 'H'
    elif np.abs(row['FTHG']- row['FTAG']) <val_draw:
        return 'D'
    else:
        return 'A'

In [2223]:
trg = target[-10:]
trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)
trg.reset_index(inplace = True, drop = True)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\5842875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)


In [2225]:
trg

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds,result
0,0,0,0.208973,0.228876,0.562151,D
1,0,0,0.698091,0.178401,0.123508,D
2,0,0,0.593220,0.228814,0.177966,D
3,0,0,0.390424,0.278389,0.331187,D
4,0,0,0.782341,0.137469,0.080190,D
5,0,0,0.814786,0.116539,0.068675,D
6,0,0,0.642160,0.204945,0.152895,D
7,0,0,0.281338,0.237574,0.481088,D
8,0,0,0.318427,0.263465,0.418108,D
9,0,0,0.282648,0.259731,0.457621,D


In [2227]:
res = pd.DataFrame(y_pred, columns = ['FTHG', 'FTAG'])
res['result'] = res.apply(choose_value2, val_draw = 0.2, axis=1)
matches = res['result'] == trg['result']

In [2263]:
over_under = pd.read_csv('over_under_1.csv')
over_under.reset_index(inplace = True, drop = True)

In [2265]:
res[['over_25', 'under_25']] = over_under[['Avg>2.5', 'Avg<2.5']]
res['total_g'] = res['FTHG']+ res['FTAG']
res['total_game'] = trg['FTHG']+ trg['FTAG']

In [2267]:
scr = res.loc[res['over_25']!=0]

In [2289]:
scr

,FTHG,FTAG,result,over_25,under_25,total_g,total_game,stake
0,0.914453,1.759854,A,1.58,2.24,2.674306,0,0
1,2.354356,0.796394,H,1.53,2.34,3.150750,0,0
2,1.711068,0.932975,H,1.69,2.04,2.644043,0,0
3,1.483445,1.388898,D,1.68,2.07,2.872343,0,0
4,2.667153,0.959219,H,1.39,2.78,3.626372,0,0
5,3.131036,0.887635,H,1.32,3.10,4.018671,0,0
6,2.007380,0.793473,H,1.63,2.14,2.800853,0,0
7,1.197788,1.676231,A,1.44,2.61,2.874020,0,0
8,1.444462,1.647822,A,1.56,2.28,3.092284,0,0
9,1.324367,1.373884,D,1.53,2.56,2.698251,0,0


In [2269]:
len(scr.loc[(scr['total_g'] > 2.5) & (scr['over_25']>2)])

0

In [2271]:
scr.loc[(scr['total_g'] > 2.5) & (scr['over_25']>2)]

,FTHG,FTAG,result,over_25,under_25,total_g,total_game


In [2283]:
scr.loc[(scr['total_g'] > 2.5) & (scr['over_25']>2)]['stake'].sum()*0.88

0.0

In [2285]:
len(scr.loc[(scr['total_g'] <2.5)& (scr['under_25']>2)])

0

In [2287]:
scr.loc[(scr['total_g'] <2.5)& (scr['under_25']>2)]['stake'].sum()*0.88

0.0

In [2133]:
186.83/137

1.3637226277372263

In [1832]:
scr.loc[(scr['total_g'] >2.5)]

,FTHG,FTAG,result,over_25,under_25,total_g,total_game,stake
0,2.038582,1.128980,H,1.70,2.20,3.167562,6,1.70
1,1.006065,2.194137,A,1.54,2.49,3.200201,6,1.54
2,2.428443,0.847590,H,1.80,2.04,3.276033,3,1.80
3,1.858001,1.148388,H,1.46,2.66,3.006389,1,0.00
4,1.538694,1.037012,H,1.63,2.29,2.575706,1,0.00
...,...,...,...,...,...,...,...,...
741,2.031553,1.147012,H,1.81,1.99,3.178565,0,0.00
742,2.200621,0.710368,H,1.43,2.84,2.910988,2,0.00
744,1.290492,1.388145,D,1.52,2.56,2.678637,2,0.00
747,1.475470,1.628363,D,1.68,2.22,3.103833,4,1.68


In [1830]:
scr.loc[(scr['total_g'] <2.5)]

,FTHG,FTAG,result,over_25,under_25,total_g,total_game,stake
17,1.140032,0.923943,H,1.49,2.59,2.063974,0,2.59
18,1.252346,1.091487,D,1.67,2.18,2.343832,4,0.00
22,0.992739,1.248382,A,1.65,2.27,2.241121,2,2.27
23,1.266980,1.085493,D,2.22,1.63,2.352473,3,0.00
43,1.590885,0.885842,H,1.56,2.43,2.476727,3,0.00
...,...,...,...,...,...,...,...,...
735,1.043479,1.406869,A,1.73,2.15,2.450348,1,2.15
743,1.012511,1.363134,A,1.38,3.07,2.375645,0,3.07
745,1.169188,1.163183,D,1.50,2.48,2.332371,5,0.00
746,1.326950,0.865419,H,2.10,1.76,2.192370,2,1.76


In [1728]:
202.8576/174

1.165848275862069

In [1734]:
157.5992/137

1.1503591240875912

In [1740]:
238.99040000000005/215

1.1115832558139538

In [2187]:
113/85

1.3294117647058823

In [2275]:
def money_won_goals(row):
    if row['total_g'] > 2.5:
        if row['total_game'] >2.5:
             return row['over_25']
        else:
            return 0
    else:
         if row['total_game'] <2.5:
             return row['under_25']
         else:
             return 0

In [2277]:
scr['stake'] = scr.apply(money_won_goals, axis=1)

In [2279]:
scr['stake'].sum()*0.88

0.0

In [2281]:
scr['stake']

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: stake, dtype: int64

In [2163]:
829.33/750

1.1057733333333335

In [2165]:
percentage_compatibility = matches.mean() * 100
percentage_compatibility

51.06666666666667

In [2167]:
trg['res2'] = res['result']

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\512750724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['res2'] = res['result']


In [2169]:
trg

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds,result,res2
0,3,3,0.566537,0.230440,0.203023,D,H
1,2,4,0.098600,0.175840,0.725560,A,A
2,2,1,0.719719,0.171116,0.109165,H,H
3,1,0,0.576157,0.235571,0.188272,H,H
4,1,0,0.518624,0.275270,0.206106,H,H
...,...,...,...,...,...,...,...
745,3,2,0.392395,0.292396,0.315209,H,D
746,2,0,0.486209,0.284551,0.229241,H,H
747,4,0,0.398873,0.273154,0.327973,H,D
748,1,0,0.533203,0.245773,0.221023,H,H


In [2171]:
def money_won(row):
    if row['res2'] == row['result']:
        if row['res2'] == 'H':
            return 1/row['home_win_odds']*0.88/1.05
        elif row['res2'] == 'D':
            return 1/row['draw_odds']*0.88/1.05
        else:
             return 1/row['away_win_odds']*0.88/1.05
    else:
        return 0

In [2173]:
trg['money_won'] = trg.apply(money_won, axis=1)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\32439853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['money_won'] = trg.apply(money_won, axis=1)


In [2175]:
trg['money_won'].sum()

675.2121004874898

In [1240]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators = 200, max_depth = 30)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [1241]:
predictions = model.predict(X_test)

predictions

array([[2.1119072 , 0.6883068 ],
       [3.5416553 , 1.5363808 ],
       [3.1409314 , 0.7233491 ],
       [1.5026135 , 2.1081471 ],
       [2.1060932 , 0.7686245 ],
       [2.4062266 , 1.8194178 ],
       [0.6651306 , 2.2547727 ],
       [1.6988604 , 0.75649536],
       [2.10201   , 1.1307769 ],
       [1.4181185 , 1.5403137 ],
       [1.7123829 , 1.2747993 ],
       [2.035104  , 1.837677  ],
       [1.2699234 , 1.3760148 ],
       [1.7411501 , 1.1166574 ],
       [1.3313135 , 0.959963  ],
       [1.5420438 , 1.4895061 ],
       [2.3792255 , 0.9538844 ],
       [1.3065447 , 1.703225  ],
       [1.489832  , 1.4754621 ],
       [1.8995148 , 0.69232994],
       [2.1473293 , 1.2678943 ],
       [2.110924  , 0.83068085],
       [0.957619  , 0.9745591 ],
       [3.1441724 , 1.1852651 ],
       [1.8642128 , 1.0602038 ],
       [2.3792255 , 0.9538844 ],
       [1.4003555 , 0.66764003],
       [1.8642128 , 1.0602038 ],
       [3.1441724 , 1.1852651 ],
       [0.957619  , 0.9745591 ],
       [2.

In [1242]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)

1.02508966479823

In [1243]:
pred_df = pd.DataFrame(predictions, columns = ['home_win_odds', 'draw_odds', 'away_win_odds'])
pred_df[['FTHG', 'FTAG']] = trg[['FTHG', 'FTAG']]
pred_df['probability converted'] = pred_df.apply(choose_value, axis=1)
pred_df['probability converted'].sum()

ValueError: Shape of passed values is (500, 2), indices imply (500, 3)

In [ ]:
import matplotlib.pyplot as plt
feature_importances = model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for easier manipulation
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort by importance and get the top 20 features
top_20_features = feature_importances_df.sort_values(by='Importance', ascending=False).head(20)

# Plot the top 20 feature importances
plt.figure(figsize=(10, 6))
plt.barh(top_20_features['Feature'], top_20_features['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Top 20 Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

In [ ]:
target[-500:]

In [ ]:
trg = target[-500:]

In [ ]:
def choose_value(row):
    if row['FTHG'] > row['FTAG']:
        return 'H'
    elif row['FTHG'] == row['FTAG']:
        return 'D'
    else:
        return 'A'

# Apply the function to each row
trg['result'] = trg.apply(choose_value, axis=1)

In [ ]:
trg['result']